In [1]:
# not working, but help to understand
from pytextractor import pytextractor

extractor = pytextractor.PyTextractor()

[INFO] Loaded EAST text detector 0.944140 seconds ...


In [2]:
# not use now, but could help in future
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth

def get_rows(X):
    # #############################################################################
    # Generate sample data
    #X = [1,2,4,7,9,5,4,7,9,56,57,54,60,200,297,275,243]
    X = np.reshape(X, (-1, 1))

    # #############################################################################
    # Compute clustering with MeanShift

    # The following bandwidth can be automatically detected using
    # bandwidth = estimate_bandwidth(X, quantile=0.2, n_samples=100)

    ms = MeanShift(bandwidth=None, bin_seeding=True)
    ms.fit(X)
    labels = ms.labels_
    cluster_centers = ms.cluster_centers_

    labels_unique = np.unique(labels)
    n_clusters_ = len(labels_unique)

    print("number of estimated clusters : %d" % n_clusters_)
    print(labels)
    print(X)
    return labels


In [3]:
import numpy as np
import argparse
import imutils
import cv2

def sort_contours(cnts):
	boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    # method= left-to-right and then top-to-bottom
	(cnts, boundingBoxes) = zip(* sorted ( sorted(zip(cnts, boundingBoxes),key=lambda b:b[1][0]), key=lambda b:b[1][1] ))
	# return the list of sorted contours and bounding boxes
	return (cnts, boundingBoxes)

In [4]:
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter

extracted_text = []
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\Aaron\AppData\Local\Tesseract-OCR\tesseract.exe'

def box_extraction(img_for_box_extraction_path, cropped_dir_path):
    img = cv2.imread(img_for_box_extraction_path, 0)  # Read the image
    (thresh, img_bin) = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO)  # Thresholding the image
    img_bin = 255-img_bin  # Invert the image
    cv2.imwrite("Image_bin.jpg",img_bin)
   
    # Defining a kernel length
    kernel_length = np.array(img).shape[1]//40
     
    # A verticle kernel of (1 X kernel_length), which will detect all the verticle lines from the image.
    verticle_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_length))
    # A horizontal kernel of (kernel_length X 1), which will help to detect all the horizontal line from the image.
    hori_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_length, 1))
    # A kernel of (3 X 3) ones.
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
# Morphological operation to detect verticle lines from an image
    img_temp1 = cv2.erode(img_bin, verticle_kernel, iterations=3)
    verticle_lines_img = cv2.dilate(img_temp1, verticle_kernel, iterations=3)
    cv2.imwrite("verticle_lines.jpg",verticle_lines_img)
# Morphological operation to detect horizontal lines from an image
    img_temp2 = cv2.erode(img_bin, hori_kernel, iterations=3)
    horizontal_lines_img = cv2.dilate(img_temp2, hori_kernel, iterations=3)
    cv2.imwrite("horizontal_lines.jpg",horizontal_lines_img)
# Weighting parameters, this will decide the quantity of an image to be added to make a new image.
    alpha = 0.5
    beta = 1.0 - alpha
    # This function helps to add two image with specific weight parameter to get a third image as summation of two image.
    img_final_bin = cv2.addWeighted(verticle_lines_img, alpha, horizontal_lines_img, beta, 0.0)
    img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
    (thresh, img_final_bin) = cv2.threshold(img_final_bin, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
# For Debugging
    # Enable this line to see verticle and horizontal lines in the image which is used to find boxes
    cv2.imwrite("img_final_bin.jpg",img_final_bin)
    # Find contours for image, which will detect all the boxes
    contours, hierarchy = cv2.findContours(img_final_bin, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    # Sort all the contours by top to bottom.
    (contours, boundingBoxes) = sort_contours(contours)
    idx = 0
   
    #X = [ cv2.boundingRect(c)[1] for c in contours ]
    #print([ cv2.boundingRect(c)  for c in contours ])
    #row_label = get_rows(X)
    #cv2.drawContours(img, contours, -1, (0,255,0), 3)
    #cv2.imwrite("img_final_see.jpg",img)
    row = -1

    for c in contours:
        # Returns the location and width,height for every contour
        x, y, w, h = cv2.boundingRect(c)
        
# If the box height is greater then 20, widht is >80, then only save it as a box in "cropped/" folder.
        #if (w > 80 and h > 20) and w > 3*h:
        idx += 1
     
        new_img = img[y:y+h, x:x+w]

        (thresh, new_img) = cv2.threshold(new_img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

        new_img_path = cropped_dir_path+str(idx) + '.png'
        cv2.imwrite(new_img_path, new_img)
        extracted_text.append(pytesseract.image_to_string(  Image.open(new_img_path), config ='--psm 6' ))

        #extracted_text.append(pytesseract.image_to_string(  Image.fromarray(new_img) ))

        if row != y:
            row = y
            extracted_text.append('\n') 
        else:
            extracted_text.append(',') 
       
        

box_extraction("Capture.JPG", "./Cropped/")

In [5]:
import pytesseract

text = pytesseract.image_to_string(Image.open('cropped/4.png'))
print(text)

PageNum


number of estimated clusters : 2
[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1]
